In [1]:
%load_ext autoreload
%autoreload 2
import config
import torch
from app.Classifiers.Classifier_NN import Classifier_IMGNET, Classifier_MNIST
import numpy as np
import torchvision
import matplotlib.pyplot as plt
from torch import nn
from sklearn.model_selection import train_test_split
from app.NObSP.NObSP_Decomposition import NObSP_NN_single_MultiOutput, NObSP_NN_single_MultiOutput_reg
import torch.nn.functional as F
from sklearn.manifold import TSNE
from torch.utils.data import DataLoader as tf_dataloader
import torchvision.transforms as transforms
import pandas as pd
import seaborn as sns
import os
from tqdm.notebook import tqdm
from torchvision.transforms.functional import to_pil_image
from matplotlib import colormaps
import PIL

In [2]:
# Cargar los tensores guardados
images = torch.load('images.pth')
X = torch.load('tensors_avgpooling.pth') # Convierte 'X' en un tensor si es una lista de listas
y = torch.load('labels.pth')

y = torch.tensor(y)  # Convierte 'y' en un tensor si es una lista de enteros


# Suponiendo que 'tensor_labels' es tu tensor de etiquetas
tensor_labels = y

# Paso 1: Extraer las etiquetas únicas y crear el mapeo
unique_labels = torch.unique(tensor_labels).tolist()
label_to_index = {label: index for index, label in enumerate(unique_labels)}

# Paso 2: Convertir etiquetas a índices mapeados
mapped_indices = torch.tensor([label_to_index[label.item()] for label in tensor_labels])

# Paso 3: Aplicar one-hot encoding
y = torch.nn.functional.one_hot(mapped_indices, num_classes=len(unique_labels))

In [3]:
X_sample = []
y_sample = []
index_used = []

sample_size = 400
# Supongamos que 'num_classes' es el número de clases en tus etiquetas
for i in range(10):
    indices = torch.where(y.argmax(dim=1) == i)[0]
    indices = indices[torch.randperm(len(indices))[:sample_size]]
    index_used.append(indices)
    X_sample.append(X[indices])
    y_sample.append(y[indices])

# Verifica que cada tensor en X_sample y y_sample es diferente
for i in range(len(X_sample)):
    for j in range(i + 1, len(X_sample)):
        assert id(X_sample[i]) != id(X_sample[j])
        assert id(y_sample[i]) != id(y_sample[j])

index_used = [tensor.item() for sublist in index_used for tensor in sublist]
images_sample = [images[i] for i in index_used]
X_sample = torch.cat(X_sample)
y_sample = torch.cat(y_sample)

In [4]:
N = np.size(X_sample, 0); # Defining the number of datapoints
in_feat = np.size(X_sample, 1)

In [5]:
t = np.arange(0, N)
train_split = int(0.9 * N)
p = 10

In [6]:
epochs = 300
learning_rate = 0.0001

In [7]:
# Split of the data for training and test
X_train, X_test, y_train, y_test = train_test_split(
    X_sample,
    y_sample,
    train_size = train_split,
    random_state=42
) 

## Creating model for single nonlinear effects

classifier = Classifier_MNIST(in_number=X_sample.shape[1], out_number=y_sample.shape[1]) # Creating the model
loss_fcn = nn.CrossEntropyLoss() # Definning loss function
optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)

# Training loop for the model 1

In [8]:
for epoch in range(epochs):
    classifier.train() # Setting the model in training mode
    y_prob, x_p, y_lin = classifier(X_train) #forward pass
    
    loss = loss_fcn(y_prob,
                    y_train.float())  # Compute Loss
    loss.backward() # compute backward
    optimizer.step() # update parameters
    optimizer.zero_grad() #zero grad optimizer
    
    ## Testing
    classifier.eval() # Setting the model in evalaution mode
    with torch.inference_mode():
        y_prob_pred, x_trans, y_lin_pred = classifier(X_test) # Estimating th emodel output in test data
    
    test_loss = loss_fcn(y_prob_pred,
                         y_test.float()) # Evaluating loss
    
    if epoch % 10 == 0:
        print(f'Epoch: {epoch} | Loss: {loss:.5f} | test Loss: {test_loss:.5f}') # Printing the performance of the model as it is trained

Epoch: 0 | Loss: 2.30261 | test Loss: 2.30192
Epoch: 10 | Loss: 2.28748 | test Loss: 2.28615
Epoch: 20 | Loss: 2.23666 | test Loss: 2.23288
Epoch: 30 | Loss: 2.07092 | test Loss: 2.06531
Epoch: 40 | Loss: 1.82526 | test Loss: 1.81768
Epoch: 50 | Loss: 1.67767 | test Loss: 1.68018
Epoch: 60 | Loss: 1.61838 | test Loss: 1.62455
Epoch: 70 | Loss: 1.57285 | test Loss: 1.58157
Epoch: 80 | Loss: 1.54958 | test Loss: 1.56197
Epoch: 90 | Loss: 1.53514 | test Loss: 1.55047
Epoch: 100 | Loss: 1.52577 | test Loss: 1.54436
Epoch: 110 | Loss: 1.51802 | test Loss: 1.54212
Epoch: 120 | Loss: 1.51187 | test Loss: 1.54173
Epoch: 130 | Loss: 1.50679 | test Loss: 1.54102
Epoch: 140 | Loss: 1.50219 | test Loss: 1.54019
Epoch: 150 | Loss: 1.49769 | test Loss: 1.54025
Epoch: 160 | Loss: 1.49371 | test Loss: 1.54030
Epoch: 170 | Loss: 1.49064 | test Loss: 1.53998
Epoch: 180 | Loss: 1.48825 | test Loss: 1.53950
Epoch: 190 | Loss: 1.48632 | test Loss: 1.53944
Epoch: 200 | Loss: 1.48479 | test Loss: 1.53896
Epo

In [9]:
correct = 0
total = 0

classifier.eval()
with torch.inference_mode():
    outputs, _, _ = classifier(X_test)
    for tensor, label in zip(outputs, y_test):
        predicted = torch.argmax(tensor)
        label = torch.argmax(label) 
        total += 1
        correct += (predicted == label).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.3f} %')

Test Accuracy: 92.750 %


In [10]:
classifier.eval()  # Setting the model in evaluation mode
with torch.inference_mode():
    y_prob_1, x_trans_total_1, y_est_1 = classifier(
        X_sample
    )  # Computing the nonlinear transformation of the input data X

In [11]:
betas_tensor = NObSP_NN_single_MultiOutput_reg(
    X_sample, y_est_1, classifier
)  # Computing the decomposition using NObSP. The Alpha parameters are the weigths for the Interpretation Layer
torch.save(betas_tensor, "betas_tensor_small.pth")

python(63173) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Number of tensors:   0%|          | 0/10 [00:00<?, ?it/s]

Lenght of tensors:   0%|          | 0/512 [00:00<?, ?it/s]

Lenght of tensors:   0%|          | 0/512 [00:00<?, ?it/s]

Lenght of tensors:   0%|          | 0/512 [00:00<?, ?it/s]

...

Lenght of tensors:   0%|          | 0/512 [00:00<?, ?it/s]

Lenght of tensors:   0%|          | 0/512 [00:00<?, ?it/s]

Lenght of tensors:   0%|          | 0/512 [00:00<?, ?it/s]

Lenght of tensors:   0%|          | 0/512 [00:00<?, ?it/s]

Lenght of tensors:   0%|          | 0/512 [00:00<?, ?it/s]